In [91]:
import pandas as pd
import ta 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import optuna
from sklearn.metrics import confusion_matrix

dataset = pd.read_csv("aapl_5m_train.csv")
dataset = dataset.drop(columns=['Volume', "Gmtoffset", "Timestamp", "Unnamed: 0", "Datetime"])
dataset.head()

,Open,High,Low,Close
0,133.570007,133.611602,132.389999,132.809997
1,132.750000,132.750000,131.809997,131.889999
2,131.500000,132.339996,131.500000,132.059997
3,132.000000,132.250000,131.899993,132.250000
4,132.000000,132.018096,131.520004,131.589996


In [92]:
lags = 7 

for lag in range (1, lags +1):
    dataset[f'lag_{lag}'] = dataset.Close.shift(lag)

dataset.head()

,Open,High,Low,Close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,133.570007,133.611602,132.389999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132.750000,132.750000,131.809997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN
2,131.500000,132.339996,131.500000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN
3,132.000000,132.250000,131.899993,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN
4,132.000000,132.018096,131.520004,131.589996,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN


In [93]:


dataset.head()

,Open,High,Low,Close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,133.570007,133.611602,132.389999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132.750000,132.750000,131.809997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN
2,131.500000,132.339996,131.500000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN
3,132.000000,132.250000,131.899993,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN
4,132.000000,132.018096,131.520004,131.589996,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN


In [94]:
dataset['future_return'] = dataset['Close'].shift(-24) / dataset['Close'] - 1

buy_threshold = 0.015
sell_threshold = -0.015

def generate_signal(x):
    if x > buy_threshold:
        return 'BUY'
    elif x < sell_threshold:
        return 'SELL'
    else:
        return 'WAIT'

In [95]:
dataset['signal'] = dataset['future_return'].apply(generate_signal)

In [96]:
dataset.head()

,Open,High,Low,Close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,future_return,signal
0,133.570007,133.611602,132.389999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.021836,SELL
1,132.750000,132.750000,131.809997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN,-0.016681,SELL
2,131.500000,132.339996,131.500000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN,NaN,-0.020217,SELL
3,132.000000,132.250000,131.899993,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN,NaN,-0.021450,SELL
4,132.000000,132.018096,131.520004,131.589996,132.250000,132.059997,131.889999,132.809997,NaN,NaN,NaN,-0.018554,SELL


In [97]:
def func_objective(trial):
    rsi_window = trial.suggest_int("rsi_window", 5, 20)
    ultimate_window1 = trial.suggest_int("ultimate_window1", 0, 10)
    ultimate_window2 = trial.suggest_int("ultimate_window2", 10, 20)
    ultimate_window3 = trial.suggest_int("ultimate_window3", 20, 30)
    
    williams_lbp = trial.suggest_int("williams_lbp", 10, 20)

    dataset["RSI"] = ta.momentum.RSIIndicator(dataset.Close, window=rsi_window).rsi()
    
    dataset["ultimate"] = ta.momentum.UltimateOscillator(
        high=dataset['High'], low=dataset['Low'], close=dataset['Close'],
        window1=ultimate_window1, window2=ultimate_window2, window3=ultimate_window3
    ).ultimate_oscillator()
    
    dataset['Williams'] = ta.momentum.WilliamsRIndicator(
        high=dataset['High'], low=dataset['Low'], close=dataset['Close'], lbp=williams_lbp
    ).williams_r()
    
    dataset.dropna(inplace=True)

    X = dataset.drop("signal", axis=1)
    y = dataset[["signal"]]
    index = dataset.index

    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
        X, y, index, test_size=0.2, shuffle=False
    )
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    
    C = trial.suggest_float("C", 0.01, 100, log=True)
    
    
    svm = SVC(kernel="rbf", C=C, gamma='scale', class_weight='balanced', max_iter=10_000)
    svm.fit(X_train, y_train)
    
    ypred = svm.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, ypred).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    
    return fpr
    
    

In [98]:
study = optuna.create_study(direction="minimize")
study.optimize(func_objective, n_trials=100)



[I 2025-03-25 19:53:10,173] A new study created in memory with name: no-name-796a96e4-72dd-430d-af5c-140fcf8529fd
c:\Users\AgJo413\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[W 2025-03-25 19:53:11,339] Trial 0 failed with parameters: {'rsi_window': 16, 'ultimate_window1': 9, 'ultimate_window2': 19, 'ultimate_window3': 30, 'williams_lbp': 18, 'C': 55.17883718272987} because of the following error: ValueError('too many values to unpack (expected 4)').
Traceback (most recent call last):
  File "c:\Users\AgJo413\Anaconda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\AgJo413\AppData\Local\Temp\ipykernel_31596\1950763490.py", line 43, in func_objective
    tn, fp, fn, tp = confusio

ValueError: too many values to unpack (expected 4)

In [ ]:
study.best_params

In [ ]:
study.best_value